# IO, Problem Set 1

### Name: Yifei Liu

In [1]:
## Data Process
import pandas as pd
import statsmodels.api as sm
from linearmodels.iv import IV2SLS
from linearmodels.panel.data import PanelData

In [2]:
# Load and preprocess data
data = pd.read_excel("C:/Users/一飞/Desktop/IO/cereal_data.xlsx")
data['constant'] = 1
data['ID'] = data.index

## Problem 1

### 3(a): OLS

In [5]:
# Define the independent and dependent variables
X = data[['constant', 'sugar', 'mushy', 'price']]
y = data['share']

# Estimate the OLS regression
model = sm.OLS(y, X)
results = model.fit()

# Print the summary results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  share   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     33.87
Date:                Fri, 12 May 2023   Prob (F-statistic):           2.08e-21
Time:                        16:42:57   Log-Likelihood:                 5117.8
No. Observations:                2256   AIC:                        -1.023e+04
Df Residuals:                    2252   BIC:                        -1.020e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
constant       0.0335      0.002     13.870      0.0

### 3(b): 2SLS

In [6]:
# Compute the average characteristics for all other products produced by the same firm
# sugar
def mean_sugar_without_current_product(x):
    result = pd.DataFrame()
    result['avg_sugar_except_current'] = (x['sugar'].sum() - x['sugar']) / (x['sugar'].count() - 1)
    result['ID'] = x['ID']
    return result

data_group1 = data.groupby('firm_id').apply(mean_sugar_without_current_product)
data = data.merge(data_group1, on=['firm_id', 'ID'], how='left')

# mushy
def mean_mushy_without_current_product(x):
    result = pd.DataFrame()
    result['avg_mushy_except_current'] = (x['mushy'].sum() - x['mushy']) / (x['mushy'].count() - 1)
    result['ID'] = x['ID']
    return result

data_group2 = data.groupby('firm_id').apply(mean_mushy_without_current_product)
data = data.merge(data_group2, on=['firm_id', 'ID'], how='left')

# sum sugar and mushy
data['IV_avg_same_charac_except_current'] = data['avg_mushy_except_current'] + data['avg_sugar_except_current']                          

# Define the independent variables (x_jt and p_jct)
X = data[['constant', 'sugar', 'mushy', 'price']]
y = data['share']
instrument = data[['IV_avg_same_charac_except_current']]

# Estimate the 2SLS regression (Method; see Method 2 in the twoSLS_rivals section)
model = IV2SLS(dependent=y, exog=X[['constant', 'sugar', 'mushy']], endog=X['price'], instruments=instrument['IV_avg_same_charac_except_current'])
results = model.fit()

# Print the summary results
print(results.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  share   R-squared:                     -0.0582
Estimator:                    IV-2SLS   Adj. R-squared:                -0.0596
No. Observations:                2256   F-statistic:                    27.383
Date:                Fri, May 12 2023   P-value (F-stat)                0.0000
Time:                        16:44:24   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
constant      -0.0004     0.0141    -0.0263     0.9790     -0.0281      0.0274
sugar          0.0003     0.0002     1.4396     0.15

### 3(c): 2SLS

In [7]:
# Compute the average characteristics for all other products produced by the rivals firm

# sugar
def mean_sugar_without_current_firm(x):
    avg_sugar = x['sugar'][x.index != x.name].mean()
    result = pd.DataFrame({'avg_sugar_except_firm': avg_sugar}, index=x.index)
    return result

data_group1 = data.groupby('firm_id').apply(mean_sugar_without_current_firm).reset_index(level='firm_id', drop=True)
data = data.join(data_group1, on='ID')

# mushy
def mean_mushy_without_current_firm(x):
    avg_mushy = x['mushy'][x.index != x.name].mean()
    result = pd.DataFrame({'avg_mushy_except_firm': avg_mushy}, index=x.index)
    return result

data_group2 = data.groupby('firm_id').apply(mean_mushy_without_current_firm).reset_index(level='firm_id', drop=True)
data = data.join(data_group2, on='ID')

# sum sugar and mushy
data['IV_avg_rivals_charac_except_current'] = data['avg_sugar_except_firm'] + data['avg_mushy_except_firm']                          

# Define the independent variables (x_jt and p_jct)
X = data[['constant', 'sugar', 'mushy']]
p = data['price']
y = data['share']
instrument = data[['IV_avg_rivals_charac_except_current']]

# Estimate the 2SLS regression  (Method 2)
# First stage regression: regress price on the instrument
first_stage = sm.OLS(p, instrument).fit()
predicted_price = first_stage.predict()
predicted_price_df = pd.DataFrame(predicted_price, columns=['predicted_price'])

# Second stage regression: regress mean utility on characteristics and predicted price
second_stage = sm.OLS(y, sm.add_constant(pd.concat([X, predicted_price_df], axis=1))).fit()
print(second_stage.summary())

                            OLS Regression Results                            
Dep. Variable:                  share   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     10.39
Date:                Fri, 12 May 2023   Prob (F-statistic):           8.61e-07
Time:                        16:46:37   Log-Likelihood:                 5083.5
No. Observations:                2256   AIC:                        -1.016e+04
Df Residuals:                    2252   BIC:                        -1.014e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
constant            0.0172      0.002     